In [2]:
import pandas as pd
from pathlib import Path

school_data_to_load = Path("../Resources/schools_complete.csv")
student_data_to_load = Path("../Resources/students_complete.csv")

school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

# Local Government Area Summary

In [3]:
# Extract school names
unique_schools = school_data_complete["school_name"].nunique()
# Calculate total student numbers
total_students = school_data_complete["student_name"].count()
# Calculate total budget
total_budget = school_data["budget"].sum()
# Calculate average maths score
average_math_score = school_data_complete["maths_score"].mean()
# Calculate average reading score
average_reading_score = school_data_complete["reading_score"].mean()
# Calculate percentage of students passing maths based on pass=>50%
passing_math_score_cond = school_data_complete.loc[school_data_complete["maths_score"] >= 50]
passing_math_score_count = passing_math_score_cond["maths_score"].count()
percent_pass_math = passing_math_score_count / total_students * 100
# Calculate percentage of students passing reading based on pass=>50%
passing_reading_score_cond = school_data_complete.loc[school_data_complete["reading_score"] >= 50]
passing_reading_score_count = passing_reading_score_cond["reading_score"].count()
percent_pass_reading = passing_reading_score_count / total_students * 100
# Calculate percentage of students passing reading AND maths based on pass >=50% in both subjects
passing_both_cond = school_data_complete.loc[(school_data_complete["maths_score"] >= 50) & (school_data_complete["reading_score"] >= 50)]
passing_both_count = passing_both_cond["student_name"].count()
percent_pass_both = passing_both_count / total_students * 100
# Create data frame
area_summary = pd.DataFrame({"Total Schools": [unique_schools], "Total Students": [total_students], \
                                    "Total Budget": [total_budget], "Average Maths Score": [average_math_score], "Average Reading Score": [average_reading_score], \
                                    "% Passing Maths": [percent_pass_math], "% Passing Reading": [percent_pass_reading], \
                                    "% Overall Passing": [percent_pass_both]})
area_summary

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,39170,24649428,70.338192,69.980138,86.078632,84.426857,72.808272


# School Summary

In [4]:
# Create first data frame that contains the school name, school type, and total budget
school_summary_1 = school_data.loc[:, ["school_name", "type", "budget"]]

# Group the merged data frame by school
by_school = school_data_complete.groupby(["school_name"])

# Calculate number of student in each school
students_by_school = by_school["student_name"].count()

# Calculate avergae maths score by school
average_maths_by_school = by_school["maths_score"].mean()
# Calculate average reading score by school
average_reading_by_school = by_school["reading_score"].mean()

# Calculate percentage of students passing maths by school based on pass >=50%
passing_math_score_cond = school_data_complete.loc[school_data_complete["maths_score"] >= 50]
passing_math_by_school = passing_math_score_cond.groupby(["school_name"])
passing_math_by_school_count = passing_math_by_school["maths_score"].count()
percent_pass_math_by_school = passing_math_by_school_count / students_by_school * 100

# Calculate percentage of students passing reading by school based on pass >=50%
passing_reading_score_cond = school_data_complete.loc[school_data_complete["reading_score"] >= 50]
passing_reading_by_school = passing_reading_score_cond.groupby(["school_name"])
passing_reading_by_school_count = passing_reading_by_school["reading_score"].count()
percent_pass_reading_by_school = passing_reading_by_school_count / students_by_school * 100

# Calculate percentage of students passing maths AND reading by school based on pass >=50% in both subjects
passing_both_cond = school_data_complete.loc[(school_data_complete["maths_score"] >= 50) & (school_data_complete["reading_score"] >= 50)]
passing_both_by_school = passing_both_cond.groupby(["school_name"])
passing_both_by_school_count = passing_both_by_school["reading_score"].count()
percent_pass_both_by_school = passing_both_by_school_count / students_by_school * 100

# Create second data frame to contain the calculated values
school_summary_averages = pd.DataFrame({"Total Students": students_by_school,
                                    "Average Reading Score": average_reading_by_school,
                                    "Average Maths Score": average_maths_by_school,
                                    "% Passing Maths": percent_pass_math_by_school, 
                                    "% Passing Reading": percent_pass_reading_by_school,
                                    "% Overall Passing": percent_pass_both_by_school})

# Merge first data frame (school, type, budget) with the calculated values
per_school_summary = pd.merge(school_summary_1, school_summary_averages, how="left", on=["school_name"])

# In the merged data frame, calculate per student budget (using total budget from data frame 1 and student numbers from data frame 2)
per_school_summary["Per Student Budget"] = per_school_summary["budget"] / per_school_summary["Total Students"]

# Format the final data frame by sorting by school, resetting index, renaming and rearranging columns, and setting final index to school name column without name
per_school_summary = per_school_summary.sort_values(by="school_name")
school_summary = per_school_summary.reset_index(drop=True)
per_school_summary.rename(columns={"school_name": "", "type": "School Type", "budget": "Total School Budget"}, inplace=True)
per_school_summary = per_school_summary[["", "School Type", "Total Students", "Total School Budget", "Per Student Budget", "Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]]
per_school_summary.set_index(per_school_summary.columns[0])

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
,,,,,,,,,
Bailey High School,Government,4976,3124928,628.0,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,1081356,582.0,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,1884411,639.0,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,1763916,644.0,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,917500,625.0,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,3022020,652.0,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,248087,581.0,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,1910635,655.0,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,3094650,650.0,68.843100,69.039277,82.062592,81.978576,67.191766


# Top Performing Schools (By % Overall Passing)

In [5]:
# Sort Per School Summary in descending order and display top 5 schools
top_performing_schools = per_school_summary.sort_values("% Overall Passing", ascending=False)
top_performing_schools.head().set_index(top_performing_schools.columns[0])

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
,,,,,,,,,
Griffin High School,Independent,1468,917500,625.0,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,1081356,582.0,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,3124928,628.0,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,1049400,583.0,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,2547363,637.0,72.047762,70.935984,90.797699,87.396849,79.419855


# Bottom Performing Schools (By % Overall Passing)

In [6]:
# Sort Per School Summary in ascending order and display bottom 5 schools
bottom_performing_schools = per_school_summary.sort_values("% Overall Passing")
bottom_performing_schools.head().set_index(bottom_performing_schools.columns[0])

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
,,,,,,,,,
Hernandez High School,Government,4635,3022020,652.0,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,1910635,655.0,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,3094650,650.0,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,1319574,578.0,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,1763916,644.0,69.091274,69.572472,82.438846,82.219788,67.469880


# Maths Scores by Year

In [7]:
# Select only Year 9 data and calculate average maths scores by school
year_9 = student_data.loc[student_data["year"] == 9, :]
year_9_maths_by_school = year_9.groupby("school_name")
year_9_av_maths = year_9_maths_by_school["maths_score"].mean()

# Select only Year 10 data and calculate average maths scores by school
year_10 = student_data.loc[student_data["year"] == 10, :]
year_10_maths_by_school = year_10.groupby("school_name")
year_10_av_maths = year_10_maths_by_school["maths_score"].mean()

# Select only Year 11 data and calculate average maths scores by school
year_11 = student_data.loc[student_data["year"] == 11, :]
year_11_maths_by_school = year_11.groupby("school_name")
year_11_av_maths = year_11_maths_by_school["maths_score"].mean()

# Select only Year 12 data and calculate average maths scores by school
year_12 = student_data.loc[student_data["year"] == 12, :]
year_12_maths_by_school = year_12.groupby("school_name")
year_12_av_maths = year_12_maths_by_school["maths_score"].mean()

# Create data frame to summarise maths scores by year
maths_scores_by_year = pd.DataFrame({"Year 9": year_9_av_maths,
                                    "Year 10": year_10_av_maths,
                                    "Year 11": year_11_av_maths,
                                    "Year 12": year_12_av_maths})

# Format data frame by dropping index name, and display
maths_scores_by_year.index.name=None
maths_scores_by_year

,Year 9,Year 10,Year 11,Year 12
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393
Pena High School,71.996364,72.396000,72.523438,71.187845


# Reading Scores by Year

In [8]:
# Calculate average reading score by school based on Year 9 selection made in maths calculation
year_9_reading_by_school = year_9.groupby("school_name")
year_9_av_reading = year_9_reading_by_school["reading_score"].mean()

# Calculate average reading score by school based on Year 10 selection made in maths calculation
year_10_reading_by_school = year_10.groupby("school_name")
year_10_av_reading = year_10_reading_by_school["reading_score"].mean()

# Calculate average reading score by school based on Year 11 selection made in maths calculation
year_11_reading_by_school = year_11.groupby("school_name")
year_11_av_reading = year_11_reading_by_school["reading_score"].mean()

# Calculate average reading score by school based on Year 12 selection made in maths calculation
year_12_reading_by_school = year_12.groupby("school_name")
year_12_av_reading = year_12_reading_by_school["reading_score"].mean()

# Create data frame to summarise maths scores by year
reading_scores_by_year = pd.DataFrame({"Year 9": year_9_av_reading,
                                    "Year 10": year_10_av_reading,
                                    "Year 11": year_11_av_reading,
                                    "Year 12": year_12_av_reading})

# Format data frame by dropping index name, and display
reading_scores_by_year.index.name=None
reading_scores_by_year

,Year 9,Year 10,Year 11,Year 12
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521
Pena High School,70.949091,72.324000,71.703125,71.513812


# Scores by School Spending

In [9]:
# Take copy of Per School Summary data frame to use in the calculation
school_spending = per_school_summary.copy()

# Set spending bins, bin labels, and cut Per School Summary into bins based on the Per Student Budget column
spending_bins = [0, 585, 630, 645, 680]
spending_labels = ["<$585", "$585-630", "$630-645", "$645-680"]
school_spending["Spending Ranges (Per Student)"] = pd.cut(school_spending["Per Student Budget"], spending_bins, labels=spending_labels)

# Calculate average maths and reading scores, and percentage of students passing maths, reading, and both, and allocate to bins 
spending_maths_scores = school_spending.groupby(["Spending Ranges (Per Student)"])["Average Maths Score"].mean()
spending_reading_scores = school_spending.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean() 
spending_passing_maths = school_spending.groupby(["Spending Ranges (Per Student)"])["% Passing Maths"].mean()                                                                                   
spending_passing_reading = school_spending.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
spending_passing_both = school_spending.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()                                                                                   

# Create data frame to hold calculated values
spending_summary = pd.DataFrame({"Average Maths Score": spending_maths_scores, 
                                 "Average Reading Score": spending_reading_scores, 
                                 "% Passing Maths": spending_passing_maths, 
                                 "% Passing Reading": spending_passing_reading, 
                                 "% Overall Passing": spending_passing_both})
spending_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.364587,70.716577,88.835926,86.390517,76.721458
$585-630,72.065868,71.031297,91.518824,87.292423,79.876293
$630-645,69.854807,69.838814,84.686139,83.763585,71.004977
$645-680,68.884391,69.045403,81.568470,81.769716,66.756253


# Scores by School Size

In [10]:
# Take copy of Per School Summary data frame to use in the calculation
school_size = per_school_summary.copy()

# Set school size bins, bin labels, and cut Per School Summary into bin based on the Total Students column
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_size["School Size"] = pd.cut(school_spending["Total Students"], size_bins, labels=size_labels)
school_size

# Calculate average maths and reading scores, and percentage of students passing maths, reading, and both, and allocate to bins 
size_maths_scores = school_size.groupby(["School Size"])["Average Maths Score"].mean()
size_reading_scores = school_size.groupby(["School Size"])["Average Reading Score"].mean() 
size_passing_maths = school_size.groupby(["School Size"])["% Passing Maths"].mean()                                                                                   
size_passing_reading = school_size.groupby(["School Size"])["% Passing Reading"].mean()
size_passing_both = school_size.groupby(["School Size"])["% Overall Passing"].mean()                                                                                   

# Create data frame to hold calculated values
size_summary = pd.DataFrame({"Average Maths Score": size_maths_scores, 
                                 "Average Reading Score": size_reading_scores, 
                                 "% Passing Maths": size_passing_maths, 
                                 "% Passing Reading": size_passing_reading, 
                                 "% Overall Passing": size_passing_both})
size_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348
Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
Large (2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


# Scores by School Type

In [11]:
# Take copy of Per School Summary data frame to use in the calculation
school_type = per_school_summary.copy()

# Calculate average maths and reading scores, and percentage of students passing maths, reading, and both, by School Type
type_maths_scores = school_size.groupby(["School Type"])["Average Maths Score"].mean()
type_reading_scores = school_type.groupby(["School Type"])["Average Reading Score"].mean() 
type_passing_maths = school_type.groupby(["School Type"])["% Passing Maths"].mean()                                                                                   
type_passing_reading = school_type.groupby(["School Type"])["% Passing Reading"].mean()
type_passing_both = school_type.groupby(["School Type"])["% Overall Passing"].mean()  

# Create data frame to hold calculated values
type_summary = pd.DataFrame({"Average Maths Score": type_maths_scores, 
                                 "Average Reading Score": type_reading_scores, 
                                 "% Passing Maths": type_passing_maths, 
                                 "% Passing Reading": type_passing_reading, 
                                 "% Overall Passing": type_passing_both})
type_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340


# Analysis Findings

The present analysis looked at academic performance within and between schools within one geographical area. Average scores and passing rates in maths and reading were analysed, and overall school performance was measured based on the proportion of students passing both subjects. Schools were compared against each other based on overall passing rates as well as average scores across acacemic years. Overall student performance was also analysed based on a school's size and type classification, and its spending per student.

The results of the analysis show that school size has the largest impact on overall student performance, with small schools outperforming large schools by 9 percentage points.

Spending appears to be negatively correlated with overall student performance, with lower-spending schools outperforming higher-spending schools. The lowest spending schools outperform the highest spending schools by almost 10 percentage points.

There does not appear to be any significant difference in average performance between government and independent schools. However, government shools are overrepresented in the bottom five.

Within-school performance is reasonably consistent across academic years, suggesting that level of study is not a significant factor in a school's performance.